### General Description
#### 1. Task Statement
**Company:** Artisan Archives

**Issue:** Artisan Archives, a company specializing in the digitization and preservation of historical visual media, possesses an enormous collection of black-and-white photographs. The process of manually colorizing these images is incredibly time-consuming, costly, and requires highly skilled digital artists. This manual approach severely limits the company's ability to restore and monetize its vast archive in a timely manner.

**ML/DS Solution:** To address this, we can leverage a deep learning technique called Image-to-Image Translation using a Generative Adversarial Network (GAN). Specifically, a pix2pix model can be trained on pairs of color and grayscale images. The model learns the mapping from the grayscale input to the corresponding color output, enabling automated colorization of new images.

**Feasibility:** A manual solution is not feasible due to the sheer volume of the archive (millions of images). The cost per image and the time required for manual colorization make it commercially unviable to process the entire collection.

**Task:** Artisan Archives has hired you to develop a proof-of-concept machine learning model that can automatically colorize grayscale landscape photographs.

**Data:** The company provides the 'Landscape Image Colorization' dataset, which contains pairs of color and grayscale landscape images.

**Definition of Done:** The primary goal is to produce visually plausible colorizations. The trained model, after 10 epochs, should generate color images from grayscale inputs that are realistic and artifact-free. Success will be evaluated qualitatively through visual inspection of the output images against their ground-truth counterparts.
#### 2. Rewards
- Understanding and implementing Generative Adversarial Networks (GANs).
- Practical experience with Image-to-Image Translation (pix2pix architecture).
- Building and training models in TensorFlow and Keras.
- Implementing custom training loops for complex models.
- Data preprocessing and augmentation for computer vision tasks.
#### 3. Difficulty Level
challenging
#### 4. Task Type
Image Generation, Computer Vision, Generative Adversarial Networks
#### 5. Tools
TensorFlow, Keras, NumPy, Matplotlib, OpenCV, Scikit-learn

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import re
import time
from tqdm import tqdm
from typing import List, Tuple, Any

```json
{
  "issue": "The model requires paired color and grayscale images for training, which must be loaded from disk, preprocessed, and structured into an efficient data pipeline.",
  "action": "Implement functions to load image files from specified directories, sort them alphanumerically to ensure correct pairing, resize them to a uniform dimension (256x256), normalize pixel values to the [-1, 1] range, and then batch them into `tf.data.Dataset` objects for both training and testing sets.",
  "state": "The image data is loaded, preprocessed, and organized into training and testing `tf.data.Dataset` pipelines, ready for consumption by the model."
}
```

In [ ]:
SIZE = 256

def sorted_alphanumeric(data: List[str]) -> List[str]:
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)

def load_images(path: str, file_limit: int) -> np.ndarray:
    images = []
    files = os.listdir(path)
    files = sorted_alphanumeric(files)
    for i in tqdm(files):
        if len(images) >= file_limit:
            break
        img = cv2.imread(os.path.join(path, i), 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0
        images.append(keras.preprocessing.image.img_to_array(img))
    return np.array(images)

def create_datasets(color_images: np.ndarray, gray_images: np.ndarray, train_size: int, batch_size: int) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
    train_color_ds = tf.data.Dataset.from_tensor_slices(color_images[:train_size]).batch(batch_size)
    train_gray_ds = tf.data.Dataset.from_tensor_slices(gray_images[:train_size]).batch(batch_size)
    
    test_color_ds = tf.data.Dataset.from_tensor_slices(color_images[train_size:]).batch(batch_size)
    test_gray_ds = tf.data.Dataset.from_tensor_slices(gray_images[train_size:]).batch(batch_size)

    train_ds = tf.data.Dataset.zip((train_gray_ds, train_color_ds))
    test_ds = tf.data.Dataset.zip((test_gray_ds, test_color_ds))

    return train_ds, test_ds

# NOTE: The original notebook had hardcoded paths. Update these to your local environment.
color_path = '../input/landscape-image-colorization/landscape Images/color'
gray_path = '../input/landscape-image-colorization/landscape Images/gray'
IMAGE_LIMIT = 2200
TRAIN_SPLIT = 2000
BATCH_SIZE = 64

color_img_array = load_images(color_path, IMAGE_LIMIT)
gray_img_array = load_images(gray_path, IMAGE_LIMIT)

train_dataset, test_dataset = create_datasets(color_img_array, gray_img_array, TRAIN_SPLIT, BATCH_SIZE)

```json
{
  "issue": "Before training, it's crucial to verify that the data has been loaded and paired correctly.",
  "action": "Create a utility function that fetches a few sample pairs from the dataset and displays the grayscale input and its corresponding color ground truth side-by-side.",
  "state": "Visual confirmation is obtained, showing that the input data and target data are correctly aligned and formatted."
}
```

In [ ]:
def visualize_sample_data(dataset: tf.data.Dataset, num_samples: int = 3):
    for gray_batch, color_batch in dataset.take(num_samples):
        plt.figure(figsize=(10, 5))
        
        plt.subplot(1, 2, 1)
        plt.title('Grayscale Input')
        plt.imshow(gray_batch[0])
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title('Color Ground Truth')
        plt.imshow(color_batch[0])
        plt.axis('off')
        
        plt.show()

visualize_sample_data(train_dataset)

```json
{
  "issue": "The core of the pix2pix model requires a Generator and a Discriminator. The Generator must learn to create realistic color images, while the Discriminator must learn to distinguish real color images from fake ones.",
  "action": "Define three functions: `build_generator` creates a U-Net architecture, which is excellent for image-to-image tasks as it preserves spatial information through skip connections. `build_discriminator` creates a PatchGAN classifier, which evaluates realism on patches of the image rather than the whole, promoting sharper outputs. `downsample` and `upsample` utility functions are also created to build these models cleanly.",
  "state": "The architectural blueprints for the Generator and Discriminator are complete, and instances of these models are created and ready for training."
}
```

In [ ]:
def downsample(filters: int, size: int, apply_batchnorm: bool = True) -> keras.Sequential:
    result = keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False))
    if apply_batchnorm:
        result.add(layers.BatchNormalization())
    result.add(layers.LeakyReLU())
    return result

def upsample(filters: int, size: int, apply_dropout: bool = False) -> keras.Sequential:
    result = keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2, padding='same', kernel_initializer='he_normal', use_bias=False))
    result.add(layers.BatchNormalization())
    if apply_dropout:
        result.add(layers.Dropout(0.5))
    result.add(layers.ReLU())
    return result

def build_generator() -> keras.Model:
    inputs = layers.Input(shape=[256, 256, 3])
    down_stack = [
        downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
        downsample(128, 4), # (bs, 64, 64, 128)
        downsample(256, 4), # (bs, 32, 32, 256)
        downsample(512, 4), # (bs, 16, 16, 512)
        downsample(512, 4), # (bs, 8, 8, 512)
        downsample(512, 4), # (bs, 4, 4, 512)
        downsample(512, 4), # (bs, 2, 2, 512)
        downsample(512, 4), # (bs, 1, 1, 512)
    ]
    up_stack = [
        upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
        upsample(512, 4), # (bs, 16, 16, 1024)
        upsample(256, 4), # (bs, 32, 32, 512)
        upsample(128, 4), # (bs, 64, 64, 256)
        upsample(64, 4), # (bs, 128, 128, 128)
    ]
    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(3, 4, strides=2, padding='same', kernel_initializer=initializer, activation='tanh')
    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = reversed(skips[:-1])
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])
    x = last(x)
    return keras.Model(inputs=inputs, outputs=x)

def build_discriminator() -> keras.Model:
    initializer = tf.random_normal_initializer(0., 0.02)
    inp = layers.Input(shape=[256, 256, 3], name='input_image')
    tar = layers.Input(shape=[256, 256, 3], name='target_image')
    x = layers.concatenate([inp, tar])
    down1 = downsample(64, 4, False)(x)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)
    zero_pad1 = layers.ZeroPadding2D()(down3)
    conv = layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer, use_bias=False)(zero_pad1)
    batchnorm1 = layers.BatchNormalization()(conv)
    leaky_relu = layers.LeakyReLU()(batchnorm1)
    zero_pad2 = layers.ZeroPadding2D()(leaky_relu)
    last = layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2)
    return keras.Model(inputs=[inp, tar], outputs=last)

generator = build_generator()
discriminator = build_discriminator()
generator.summary()
discriminator.summary()

```json
{
  "issue": "Training a GAN requires a carefully defined set of loss functions and optimizers, as well as a single-step training function that correctly updates both the generator and discriminator.",
  "action": "Define separate loss functions for the generator and discriminator. The discriminator loss penalizes misclassifying real and fake images. The generator loss has two components: a GAN loss to fool the discriminator and an L1 loss to ensure the generated image is structurally similar to the ground truth. An Adam optimizer is created for each model. Finally, the `@tf.function`-decorated `train_step` function is created to perform one step of training: it calculates losses, computes gradients, and applies them to update the model weights.",
  "state": "The complete logic for a single training step, including loss calculations and model updates, is encapsulated and optimized, ready to be called in a loop."
}
```

In [ ]:
LAMBDA = 100
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

def generator_loss(disc_generated_output: tf.Tensor, gen_output: tf.Tensor, target: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    total_gen_loss = gan_loss + (LAMBDA * l1_loss)
    return total_gen_loss, gan_loss, l1_loss

def discriminator_loss(disc_real_output: tf.Tensor, disc_generated_output: tf.Tensor) -> tf.Tensor:
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss

@tf.function
def train_step(input_image: tf.Tensor, target: tf.Tensor):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(input_image, training=True)
        disc_real_output = discriminator([input_image, target], training=True)
        disc_generated_output = discriminator([input_image, gen_output], training=True)
        gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)
    
    generator_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

```json
{
  "issue": "The model must be trained for a set number of epochs over the entire training dataset.",
  "action": "A `fit` function is defined to manage the training loop. It iterates for a specified number of epochs, and in each epoch, it iterates through every batch in the training dataset, calling the `train_step` function for each. It also times each epoch to monitor training speed.",
  "state": "The GAN model is trained on the landscape dataset for 10 epochs. The weights of the generator and discriminator are updated, and the model learns to perform the colorization task."
}
```

In [ ]:
def fit(train_ds: tf.data.Dataset, epochs: int):
    for epoch in range(epochs):
        start = time.time()
        print(f"Epoch: {epoch + 1}/{epochs}")
        
        for n, (input_image, target) in tqdm(enumerate(train_ds), total=len(list(train_ds.as_numpy_iterator()))):
            train_step(input_image, target)
        
        print(f'Time taken for epoch {epoch + 1} is {time.time()-start:.2f} sec\n')

EPOCHS = 10
fit(train_dataset, epochs=EPOCHS)

```json
{
  "issue": "After training, the model's performance must be visually assessed to determine the quality of the colorization.",
  "action": "An evaluation function is created that takes the trained generator and a sample from the test set. It generates a colorized image from the grayscale input and then plots the input, the ground truth, and the model's prediction side-by-side for easy comparison.",
  "state": "The qualitative performance of the model is demonstrated through several plotted examples, showing its ability to generate plausible color images from grayscale inputs."
}
```

In [ ]:
def evaluate_and_plot_results(model: keras.Model, test_dataset: tf.data.Dataset, num_images: int = 3):
    for example_input, example_target in test_dataset.take(num_images):
        prediction = model(example_input, training=True)
        
        plt.figure(figsize=(15, 5))
        display_list = [example_input[0], example_target[0], prediction[0]]
        title = ['Input Image', 'Ground Truth', 'Predicted Image']
        
        for i in range(3):
            plt.subplot(1, 3, i + 1)
            plt.title(title[i])
            # Clip values to [0, 1] for proper display
            image_to_show = np.clip(display_list[i], 0, 1)
            plt.imshow(image_to_show)
            plt.axis('off')
        plt.show()

evaluate_and_plot_results(generator, test_dataset)